<a href="https://colab.research.google.com/github/Nima-Nilchian/stress-detection/blob/master/stress_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
import torch
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from google.colab import drive

In [ ]:
!pip install kaggle

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = ""
os.environ['KAGGLE_KEY'] = ""
!kaggle datasets download -d ruchi798/stress-analysis-in-social-media


  0% 0.00/1.31M [00:00<?, ?B/s]
100% 1.31M/1.31M [00:00<00:00, 140MB/s]


In [ ]:
drive.mount('/content/drive')
!unzip -o stress-analysis-in-social-media -d '/content/drive/MyDrive/kaggle/'


Mounted at /content/drive
Archive:  stress-analysis-in-social-media.zip
  inflating: /content/drive/MyDrive/kaggle/dreaddit-test.csv  
  inflating: /content/drive/MyDrive/kaggle/dreaddit-train.csv  


In [ ]:
csv_file_path = '/content/drive/MyDrive/kaggle/dreaddit-train.csv'
data = pd.read_csv(csv_file_path)
data

,subreddit,post_id,sentence_range,text,id,label,confidence,social_timestamp,social_karma,syntax_ari,...,lex_dal_min_pleasantness,lex_dal_min_activation,lex_dal_min_imagery,lex_dal_avg_activation,lex_dal_avg_imagery,lex_dal_avg_pleasantness,social_upvote_ratio,social_num_comments,syntax_fk_grade,sentiment
0,ptsd,8601tu,"(15, 20)","He said he had not felt that way before, sugge...",33181,1,0.800000,1521614353,5,1.806818,...,1.0000,1.1250,1.0,1.77000,1.52211,1.89556,0.86,1,3.253573,-0.002742
1,assistance,8lbrx9,"(0, 5)","Hey there r/assistance, Not sure if this is th...",2606,0,1.000000,1527009817,4,9.429737,...,1.1250,1.0000,1.0,1.69586,1.62045,1.88919,0.65,2,8.828316,0.292857
2,ptsd,9ch1zh,"(15, 20)",My mom then hit me with the newspaper and it s...,38816,1,0.800000,1535935605,2,7.769821,...,1.0000,1.1429,1.0,1.83088,1.58108,1.85828,0.67,0,7.841667,0.011894
3,relationships,7rorpp,"[5, 10]","until i met my new boyfriend, he is amazing, h...",239,1,0.600000,1516429555,0,2.667798,...,1.0000,1.1250,1.0,1.75356,1.52114,1.98848,0.50,5,4.104027,0.141671
4,survivorsofabuse,9p2gbc,"[0, 5]",October is Domestic Violence Awareness Month a...,1421,1,0.800000,1539809005,24,7.554238,...,1.0000,1.1250,1.0,1.77644,1.64872,1.81456,1.00,1,7.910952,-0.204167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2833,relationships,7oee1t,"[35, 40]","* Her, a week ago: Precious, how are you? (I i...",1713,0,1.000000,1515187044,13,-1.369333,...,1.4000,1.0000,1.0,1.71133,1.45301,2.00304,0.84,16,0.254444,0.552066
2834,ptsd,9p4ung,"[20, 25]",I don't have the ability to cope with it anymo...,1133,1,1.000000,1539827412,33,9.425478,...,1.0000,1.0000,1.0,1.65003,1.56842,1.81527,0.96,6,8.640664,-0.220370
2835,anxiety,9nam6l,"(5, 10)",In case this is the first time you're reading ...,10442,0,1.000000,1539269312,2,11.060675,...,1.1250,1.1250,1.0,1.79768,1.49074,1.92286,1.00,1,9.951524,0.045455
2836,almosthomeless,5y53ya,"[5, 10]",Do you find this normal? They have a good rela...,1834,0,0.571429,1488938143,4,2.421912,...,1.1111,1.1429,1.0,1.71642,1.57627,1.89972,0.75,7,4.036765,0.159722


In [ ]:
!pip install -U transformers
!pip install datasets evaluate
!pip install git+https://github.com/huggingface/accelerate
!pip install huggingface-hub
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.4 MB/s eta 0:00:00


In [ ]:
# !huggingface-cli login
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
import evaluate
from datasets import load_dataset
import numpy as np
import torch

In [ ]:
data_files = {"train": "dreaddit-train.csv", "test": "dreaddit-test.csv"}
dataset = load_dataset("/content/drive/MyDrive/kaggle/", data_files=data_files)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
keep = ['text', 'label']
selected_dataset = dataset.select_columns(keep)

In [ ]:
selected_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 2838
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 715
    })
})

In [ ]:
split_ratio = 0.8

split_index = int(selected_dataset['train'].num_rows * split_ratio)
train_dataset = selected_dataset['train'].select(range(split_index))
eval_dataset = selected_dataset['train'].select(range(split_index, dataset['train'].num_rows))

In [ ]:
train_dataset


Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2270
})

In [ ]:
model_name = "mental/mental-bert-base-uncased"
# model_name = "mental/mental-roberta-base"
# model_name = "bert-base-uncased"
# model_name = "roberta-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)

def encode(examples):
    return tokenizer(examples['text'], padding=True, truncation=True)

train_dataset = train_dataset.map(encode, batched=True)
eval_dataset = eval_dataset.map(encode, batched=True)

Map:   0%|          | 0/2270 [00:00<?, ? examples/s]

Map:   0%|          | 0/568 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2,
                                          id2label=id2label, label2id=label2id)

device = 'cuda' if torch.cuda.is_available else 'cpu'
print(device)
model = model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at mental/mental-bert-base-uncased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


cuda


In [ ]:
f1 = evaluate.load("f1")
# accuracy = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments(
    output_dir="mental_bert_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to='tensorboard',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= train_dataset,
    eval_dataset= eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,No log,0.404687,0.848185


TrainOutput(global_step=142, training_loss=0.28749901140239875, metrics={'train_runtime': 187.1221, 'train_samples_per_second': 12.131, 'train_steps_per_second': 0.759, 'total_flos': 440947406566800.0, 'train_loss': 0.28749901140239875, 'epoch': 1.0})

In [ ]:
test_dataset = dataset['test'].map(encode, batched=True)
preds = trainer.predict(test_dataset)

true_labels = preds[1]

y_pred_transformers = np.argmax(preds[0], -1)

Map:   0%|          | 0/715 [00:00<?, ? examples/s]

In [ ]:
%load_ext tensorboard
%tensorboard --logdir 'content/runs'

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

f1_score(true_labels, y_pred_transformers)

0.8169761273209549

In [ ]:
## Scores

# 83.2 mental_roberta
# 80.5 bert
# 82.18 roberta
# 81.7 mental-bert